In [68]:
import nltk
from nltk.corpus import names
import random
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import KFold
import numpy as np


nltk.download("names")



[nltk_data] Downloading package names to /Users/lailson/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [6]:
#Mostar 10 primeiros nomes maculinos e femininos
nomes_masculinos = names.words("male.txt")
nomes_femininos = names.words("female.txt")

print("Primeiros 10 nomes masculinos:", nomes_masculinos[:10])
print("Primeiros 10 nomes femininos:", nomes_femininos[:10])

Primeiros 10 nomes masculinos: ['Aamir', 'Aaron', 'Abbey', 'Abbie', 'Abbot', 'Abbott', 'Abby', 'Abdel', 'Abdul', 'Abdulkarim']
Primeiros 10 nomes femininos: ['Abagael', 'Abagail', 'Abbe', 'Abbey', 'Abbi', 'Abbie', 'Abby', 'Abigael', 'Abigail', 'Abigale']


In [7]:
#Mostrar tamanho do corpus
print("Total de nomes masculinos:", len(nomes_masculinos))
print("Total de nomes femininos:", len(nomes_femininos))


Total de nomes masculinos: 2943
Total de nomes femininos: 5001


In [8]:
# Mostrar nomes aleatórios
nomes_rotulados = ([(nome, 'masculino') for nome in nomes_masculinos] +
                   [(nome, 'feminino') for nome in nomes_femininos])


import random
random.shuffle(nomes_rotulados)
print("Primeiros 10 nomes rotulados:", nomes_rotulados[:10])


Primeiros 10 nomes rotulados: [('Uriah', 'masculino'), ('Kalman', 'masculino'), ('Nevin', 'masculino'), ('Hunt', 'masculino'), ('Olympia', 'feminino'), ('Kimberly', 'feminino'), ('Nichole', 'feminino'), ('Jacky', 'feminino'), ('Jacqueline', 'feminino'), ('Gail', 'masculino')]


In [9]:

# Função para extrair features
def extrair_features(nome):
    features = {
        'ultima_letra': nome[-1].lower(),
        'primeira_letra': nome[0].lower(),
        'tamanho_nome': len(nome),
        'duas_ultimas_letras': nome[-2:].lower(),
        'segunda_letra': nome[1].lower() if len(nome) > 1 else '',                 
        'duas_primeiras_letras': nome[:2].lower(),                                 
        'numero_de_vogais': sum(1 for c in nome if c.lower() in 'aeiou'),           
        'numero_de_consoantes': sum(1 for c in nome if c.lower() not in 'aeiou'),   
        'quantidade_letras_repetidas': sum(nome.lower().count(c) > 1 for c in set(nome.lower())),
        'termina_com_vogal': nome[-1].lower() in 'aeiou',                          
        'quantidade_letras_unicas': len(set(nome.lower())),                        
    }
    return features


conjunto_features = [(extrair_features(nome), genero) for (nome, genero) in nomes_rotulados]
conjunto_features[0]


({'ultima_letra': 'h',
  'primeira_letra': 'u',
  'tamanho_nome': 5,
  'duas_ultimas_letras': 'ah',
  'segunda_letra': 'r',
  'duas_primeiras_letras': 'ur',
  'numero_de_vogais': 3,
  'numero_de_consoantes': 2,
  'quantidade_letras_repetidas': 0,
  'termina_com_vogal': False,
  'quantidade_letras_unicas': 5},
 'masculino')

In [10]:

# Proporcões
proporcoes = [0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95] 
resultados = []

for proporcao in proporcoes:
    tamanho_treino = int(proporcao * len(conjunto_features))
    conjunto_treino, conjunto_teste = conjunto_features[:tamanho_treino], conjunto_features[tamanho_treino:]
    
    classificador = NaiveBayesClassifier.train(conjunto_treino)
    
    y_true = [genero for (features, genero) in conjunto_teste]
    y_pred = [classificador.classify(features) for (features, genero) in conjunto_teste]
    
    acuracia = accuracy(classificador, conjunto_teste)
    
    precisao = precision_score(y_true, y_pred, pos_label='feminino')
    recall = recall_score(y_true, y_pred, pos_label='feminino')
    f1 = f1_score(y_true, y_pred, pos_label='feminino')
    
    resultados.append((proporcao, acuracia, precisao, recall, f1))
    
    print(f"Proporção de Treino/Teste: {int(proporcao * 100)}/{int((1 - proporcao) * 100)}%")
    print(f"Acurácia: {acuracia:.4f}, Precisão: {precisao:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}\n")

Proporção de Treino/Teste: 50/50%
Acurácia: 0.7812, Precisão: 0.8702, Recall: 0.7706, F1: 0.8174

Proporção de Treino/Teste: 60/40%
Acurácia: 0.7788, Precisão: 0.8695, Recall: 0.7677, F1: 0.8154

Proporção de Treino/Teste: 70/30%
Acurácia: 0.7697, Precisão: 0.8617, Recall: 0.7584, F1: 0.8068

Proporção de Treino/Teste: 75/25%
Acurácia: 0.7714, Precisão: 0.8600, Recall: 0.7627, F1: 0.8084

Proporção de Treino/Teste: 80/19%
Acurácia: 0.7659, Precisão: 0.8506, Recall: 0.7600, F1: 0.8028

Proporção de Treino/Teste: 85/15%
Acurácia: 0.7743, Precisão: 0.8679, Recall: 0.7615, F1: 0.8112

Proporção de Treino/Teste: 90/9%
Acurácia: 0.7786, Precisão: 0.8756, Recall: 0.7665, F1: 0.8174

Proporção de Treino/Teste: 95/5%
Acurácia: 0.7889, Precisão: 0.8816, Recall: 0.7791, F1: 0.8272



Com a propoção de 95% para treino e 5% para teste tivemos a melhor acurácia de 0.7990.
Vamos aplicar a validação cruzada para verificar se a proporção se mantem.

In [11]:
# Avaliação para cada proporção usando K-Fold Cross-Validation
for proporcao in proporcoes:
    print(f"\nProporção de Treino/Teste: {int(proporcao * 100)}/{int((1 - proporcao) * 100)}%")
    
    tamanho_treino = int(proporcao * len(conjunto_features))
    conjunto_treino = conjunto_features[:tamanho_treino]
    conjunto_teste = conjunto_features[tamanho_treino:]
    kf = KFold(n_splits=10, shuffle=True, random_state=42)  

    
    acuracias = []
    precisoes = []
    recalls = []
    f1_scores = []

    for treino_index, teste_index in kf.split(conjunto_treino):
        fold_treino = [conjunto_treino[i] for i in treino_index]
        fold_teste = [conjunto_treino[i] for i in teste_index]
        classificador = NaiveBayesClassifier.train(fold_treino)

        y_true = [genero for (_, genero) in fold_teste]
        y_pred = [classificador.classify(features) for (features, _) in fold_teste]

        acuracia = accuracy(classificador, fold_teste)
        precisao = precision_score(y_true, y_pred, pos_label='feminino')
        recall = recall_score(y_true, y_pred, pos_label='feminino')
        f1 = f1_score(y_true, y_pred, pos_label='feminino')

        acuracias.append(acuracia)
        precisoes.append(precisao)
        recalls.append(recall)
        f1_scores.append(f1)

    print(f"Acurácia Média: {np.mean(acuracias):.4f}")
    print(f"Precisão Média: {np.mean(precisoes):.4f}")
    print(f"Recall Médio: {np.mean(recalls):.4f}")
    print(f"F1-Score Médio: {np.mean(f1_scores):.4f}")



Proporção de Treino/Teste: 50/50%
Acurácia Média: 0.7777
Precisão Média: 0.8560
Recall Médio: 0.7731
F1-Score Médio: 0.8122

Proporção de Treino/Teste: 60/40%
Acurácia Média: 0.7747
Precisão Média: 0.8591
Recall Médio: 0.7648
F1-Score Médio: 0.8091

Proporção de Treino/Teste: 70/30%
Acurácia Média: 0.7797
Precisão Média: 0.8662
Recall Médio: 0.7676
F1-Score Médio: 0.8139

Proporção de Treino/Teste: 75/25%
Acurácia Média: 0.7803
Precisão Média: 0.8653
Recall Médio: 0.7706
F1-Score Médio: 0.8150

Proporção de Treino/Teste: 80/19%
Acurácia Média: 0.7808
Precisão Média: 0.8671
Recall Médio: 0.7707
F1-Score Médio: 0.8158

Proporção de Treino/Teste: 85/15%
Acurácia Média: 0.7787
Precisão Média: 0.8641
Recall Médio: 0.7688
F1-Score Médio: 0.8135

Proporção de Treino/Teste: 90/9%
Acurácia Média: 0.7790
Precisão Média: 0.8658
Recall Médio: 0.7666
F1-Score Médio: 0.8130

Proporção de Treino/Teste: 95/5%
Acurácia Média: 0.7786
Precisão Média: 0.8668
Recall Médio: 0.7659
F1-Score Médio: 0.8129


A proporção 70/30 apresentou o melhor acurácia média 0.7793 e também oo melhor f1-score de 0.8160

Sem validação cruzada: 95/5
Com validação cruzada: 70/30

In [12]:
# Mostrar as features mais informativas
print("\nFeatures mais informativas:")
classificador.show_most_informative_features(20)


Features mais informativas:
Most Informative Features
     duas_ultimas_letras = 'na'           femini : mascul =     89.1 : 1.0
     duas_ultimas_letras = 'la'           femini : mascul =     70.1 : 1.0
     duas_ultimas_letras = 'ia'           femini : mascul =     50.8 : 1.0
            ultima_letra = 'a'            femini : mascul =     34.9 : 1.0
     duas_ultimas_letras = 'sa'           femini : mascul =     32.0 : 1.0
     duas_ultimas_letras = 'ta'           femini : mascul =     30.7 : 1.0
            ultima_letra = 'k'            mascul : femini =     30.7 : 1.0
     duas_ultimas_letras = 'io'           mascul : femini =     26.0 : 1.0
     duas_ultimas_letras = 'ra'           femini : mascul =     24.0 : 1.0
     duas_ultimas_letras = 'us'           mascul : femini =     23.9 : 1.0
     duas_ultimas_letras = 'do'           mascul : femini =     23.8 : 1.0
     duas_ultimas_letras = 'rd'           mascul : femini =     22.7 : 1.0
     duas_ultimas_letras = 'ld'           mas

In [13]:
nome_novo = "Lailson"
features_nome_novo = extrair_features(nome_novo)

genero_previsto = classificador.classify(features_nome_novo)
print(f"O gênero previsto para o nome '{nome_novo}' é: {genero_previsto}")


O gênero previsto para o nome 'Lailson' é: masculino


In [14]:
nome_novo2 = "Henrique"
features_nome_novo2 = extrair_features(nome_novo2)

genero_previsto = classificador.classify(features_nome_novo2)
print(f"O gênero previsto para o nome '{nome_novo2}' é: {genero_previsto}")

O gênero previsto para o nome 'Henrique' é: feminino


In [64]:
# Brail - fEMININO
with open("dataset/femininoBR.txt", "r") as file:
    nomes_femininos_br = {linha.strip() for linha in file}  # Usando conjunto para evitar duplicados

acertos_feminino_br = 0
erros_feminino_br = 0

for nome in nomes_femininos_br:
    features_nome = extrair_features(nome)  
    genero_previsto = classificador.classify(features_nome)

    if genero_previsto == "feminino":
        acertos_feminino_br += 1
    else:
        erros_feminino_br += 1
        print(f"Erro: Nome {nome} classificado como {genero_previsto}")

total_nomes_feminino_br = len(nomes_femininos_br)
acuracia_feminino_br = acertos_feminino_br / total_nomes_feminino_br

print(f"\nTotal de nomes femininos brasileiros testados: {total_nomes_feminino_br}")
print(f"Acertos em nomes femininos brasileiros: {acertos_feminino_br}")
print(f"Erros em nomes femininos brasileiros: {erros_feminino_br}")
print(f"Acurácia em nomes femininos brasileiros: {acuracia_feminino_br:.4f}")


Erro: Nome Isabelly classificado como masculino
Erro: Nome Elis classificado como masculino
Erro: Nome Liz classificado como masculino
Erro: Nome Sarah classificado como masculino
Erro: Nome Esther classificado como masculino
Erro: Nome Eloá classificado como masculino
Erro: Nome Beatriz classificado como masculino
Erro: Nome Laís classificado como masculino
Erro: Nome Isis classificado como masculino
Erro: Nome Isabel classificado como masculino
Erro: Nome Yasmin classificado como masculino
Erro: Nome Ana Liz classificado como masculino
Erro: Nome Maitê classificado como masculino

Total de nomes femininos brasileiros testados: 100
Acertos em nomes femininos brasileiros: 87
Erros em nomes femininos brasileiros: 13
Acurácia em nomes femininos brasileiros: 0.8700


In [63]:
# Brasil - maculino
with open("dataset/masculinoBR.txt", "r") as file:
    nomes_masculinos_br = {linha.strip() for linha in file}  

acertos_masculino_br = 0
erros_masculino_br = 0

for nome in nomes_masculinos_br:
    features_nome = extrair_features(nome)  
    genero_previsto = classificador.classify(features_nome) 

    if genero_previsto == "masculino":
        acertos_masculino_br += 1
    else:
        erros_masculino_br += 1
        print(f"Errou: Nome {nome} classificado como {genero_previsto}")

total_nomes_masculino_br = len(nomes_masculinos_br)
acuracia_masculino_br = acertos_masculino_br / total_nomes_masculino_br

print(f"\nTotal de nomes masculinos brasileiros testados: {total_nomes_masculino_br}")
print(f"Acertos em nomes masculinos brasileiros: {acertos_masculino_br}")
print(f"Erros em nomes masculinos brasileiros: {erros_masculino_br}")
print(f"Acurácia em nomes masculinos brasileiros: {acuracia_masculino_br:.4f}")


Errou: Nome Henrique classificado como feminino
Errou: Nome Levi classificado como feminino
Errou: Nome João Miguel classificado como feminino
Errou: Nome Davi classificado como feminino
Errou: Nome Asafe classificado como feminino
Errou: Nome Davi Lucca classificado como feminino
Errou: Nome Ravi Lucca classificado como feminino
Errou: Nome Emanuel classificado como feminino
Errou: Nome Felipe classificado como feminino
Errou: Nome Luiz Miguel classificado como feminino
Errou: Nome Anthony classificado como feminino
Errou: Nome João Guilherme classificado como feminino
Errou: Nome Yuri classificado como feminino
Errou: Nome Kalel classificado como feminino
Errou: Nome Lucca classificado como feminino
Errou: Nome Kauê classificado como feminino
Errou: Nome Vicente classificado como feminino
Errou: Nome Guilherme classificado como feminino
Errou: Nome Luiz Henrique classificado como feminino
Errou: Nome Estevão classificado como feminino
Errou: Nome Pedro Henrique classificado como femi

In [61]:
# Alemanha
with open("dataset/femininoDE.txt", "r") as file:
    nomes_femininos_de = {(linha.strip(), "feminino") for linha in file}  

with open("dataset/masculinoDE.txt", "r") as file:
    nomes_masculinos_de = {(linha.strip(), "masculino") for linha in file}  

dataset_alemanha = list(nomes_femininos_de | nomes_masculinos_de)  

acertos_alemanha = 0
erros_alemanha = 0

for nome, genero_real in dataset_alemanha:
    features_nome = extrair_features(nome)  
    genero_previsto = classificador.classify(features_nome) 

    if genero_previsto == genero_real:
        acertos_alemanha += 1
    else:
        erros_alemanha += 1
        print(f"Erro: Nome {nome} classificado como {genero_previsto}, esperado: {genero_real}")

total_nomes_alemanha = len(dataset_alemanha)
acuracia_alemanha = acertos_alemanha / total_nomes_alemanha

print(f"\nTotal de nomes testados na Alemanha: {total_nomes_alemanha}")
print(f"Acertos na Alemanha: {acertos_alemanha}")
print(f"Erros na Alemanha: {erros_alemanha}")
print(f"Acurácia na Alemanha: {acuracia_alemanha:.4f}")


Erro: Nome Gertrud classificado como masculino, esperado: feminino
Erro: Nome Ingrid classificado como masculino, esperado: feminino
Erro: Nome Luca classificado como feminino, esperado: masculino
Erro: Nome Maximilian classificado como feminino, esperado: masculino
Erro: Nome Karl classificado como feminino, esperado: masculino
Erro: Nome Dagmar classificado como masculino, esperado: feminino
Erro: Nome Astrid classificado como masculino, esperado: feminino
Erro: Nome Hildegard classificado como masculino, esperado: feminino
Erro: Nome Hannah classificado como masculino, esperado: feminino

Total de nomes testados na Alemanha: 95
Acertos na Alemanha: 86
Erros na Alemanha: 9
Acurácia na Alemanha: 0.9053


In [62]:
# JapÃo
with open("dataset/femininoJP.txt", "r") as file:
    nomes_femininos_jp = {(linha.strip(), "feminino") for linha in file}  

with open("dataset/masculinoJP.txt", "r") as file:
    nomes_masculinos_jp = {(linha.strip(), "masculino") for linha in file}

dataset_japao = list(nomes_femininos_jp | nomes_masculinos_jp)  

acertos_japao = 0
erros_japao = 0

for nome, genero_real in dataset_japao:
    features_nome = extrair_features(nome)  
    genero_previsto = classificador.classify(features_nome) 

    if genero_previsto == genero_real:
        acertos_japao += 1
    else:
        erros_japao += 1
        print(f"Erro: Nome {nome} classificado como {genero_previsto}, esperado: {genero_real}")

total_nomes_japao = len(dataset_japao)
acuracia_japao = acertos_japao / total_nomes_japao

print(f"\nTotal de nomes testados no Japão: {total_nomes_japao}")
print(f"Acertos no Japão: {acertos_japao}")
print(f"Erros no Japão: {erros_japao}")
print(f"Acurácia no Japão: {acuracia_japao:.4f}")


Erro: Nome Shota classificado como feminino, esperado: masculino
Erro: Nome Yuki classificado como feminino, esperado: masculino
Erro: Nome Kenji classificado como feminino, esperado: masculino
Erro: Nome Hiroshi classificado como feminino, esperado: masculino
Erro: Nome Tatsuya classificado como feminino, esperado: masculino
Erro: Nome Akiko classificado como masculino, esperado: feminino
Erro: Nome Yui classificado como masculino, esperado: feminino
Erro: Nome Satoshi classificado como feminino, esperado: masculino
Erro: Nome Chihiro classificado como masculino, esperado: feminino
Erro: Nome Sora classificado como feminino, esperado: masculino
Erro: Nome Haruki classificado como feminino, esperado: masculino
Erro: Nome Eiji classificado como feminino, esperado: masculino
Erro: Nome Daichi classificado como feminino, esperado: masculino
Erro: Nome Mao classificado como masculino, esperado: feminino
Erro: Nome Yuko classificado como masculino, esperado: feminino
Erro: Nome Ryota classi

In [65]:
# Espanha
with open("dataset/femininoES.txt", "r") as file:
    nomes_femininos_es = {(linha.strip(), "feminino") for linha in file}  

with open("dataset/masculinoES.txt", "r") as file:
    nomes_masculinos_es = {(linha.strip(), "masculino") for linha in file}  

dataset_es = list(nomes_femininos_es | nomes_masculinos_es)

acertos_espanha = 0
erros_espanha = 0

for nome, genero_real in dataset_es:
    features_nome = extrair_features(nome)  
    genero_previsto = classificador.classify(features_nome) 

    if genero_previsto == genero_real:
        acertos_espanha += 1
    else:
        erros_espanha += 1
        print(f"Erro: Nome {nome} classificado como {genero_previsto}, esperado: {genero_real}")

total_nomes_espanha = len(dataset_es)
acuracia_espanha = acertos_espanha / total_nomes_espanha

print(f"\nTotal de nomes testados em Espanha: {total_nomes_espanha}")
print(f"Acertos em Espanha: {acertos_espanha}")
print(f"Erros em Espanha: {erros_espanha}")
print(f"Acurácia em Espanha: {acuracia_espanha:.4f}")


Erro: Nome Inés classificado como masculino, esperado: feminino
Erro: Nome Beatriz classificado como masculino, esperado: feminino
Erro: Nome Esther classificado como masculino, esperado: feminino
Erro: Nome Jaime classificado como feminino, esperado: masculino
Erro: Nome Abril classificado como masculino, esperado: feminino
Erro: Nome Isabel classificado como masculino, esperado: feminino
Erro: Nome Ciro classificado como feminino, esperado: masculino
Erro: Nome Pilar classificado como masculino, esperado: feminino
Erro: Nome Jorge classificado como feminino, esperado: masculino
Erro: Nome Rocío classificado como masculino, esperado: feminino
Erro: Nome Carmen classificado como masculino, esperado: feminino

Total de nomes testados em Espanha: 83
Acertos em Espanha: 72
Erros em Espanha: 11
Acurácia em Espanha: 0.8675


In [28]:
# Portugal
with open("dataset/femininoPT.txt", "r") as file:
    nomes_femininos_pt = [(linha.strip(), "feminino") for linha in file]

with open("dataset/masculinoPT.txt", "r") as file:
    nomes_masculinos_pt = [(linha.strip(), "masculino") for linha in file]

dataset_pt = nomes_femininos_pt + nomes_masculinos_pt

acertos_portugal = 0
erros_portugal = 0

for nome, genero_real in dataset_pt:
    features_nome = extrair_features(nome)  
    genero_previsto = classificador.classify(features_nome) 

    if genero_previsto == genero_real:
        acertos_portugal += 1
    else:
        erros_portugal += 1
        print(f"Errou: Nome {nome} classificado como {genero_previsto}, esperado: {genero_real}")

total_nomes_portugal = len(dataset_pt)
acuracia_portugal = acertos_portugal / total_nomes_portugal

print(f"\nTotal de nomes testados em Portugal: {total_nomes_portugal}")
print(f"Acertos em Portugal: {acertos_portugal}")
print(f"Erros em Portugal: {erros_portugal}")
print(f"Acurácia em Portugal: {acuracia_portugal:.4f}")


Errou: Nome Isabel classificado como masculino, esperado: feminino
Errou: Nome Leonor classificado como masculino, esperado: feminino
Errou: Nome Beatriz classificado como masculino, esperado: feminino
Errou: Nome Inês classificado como masculino, esperado: feminino
Errou: Nome Beatriz classificado como masculino, esperado: feminino
Errou: Nome Inês classificado como masculino, esperado: feminino
Errou: Nome Sol classificado como masculino, esperado: feminino
Errou: Nome Iris classificado como masculino, esperado: feminino
Errou: Nome Salomé classificado como masculino, esperado: feminino
Errou: Nome Duarte classificado como feminino, esperado: masculino
Errou: Nome Henrique classificado como feminino, esperado: masculino
Errou: Nome Vicente classificado como feminino, esperado: masculino
Errou: Nome Davi classificado como feminino, esperado: masculino
Errou: Nome Emanuel classificado como feminino, esperado: masculino
Errou: Nome Guilherme classificado como feminino, esperado: masculi

In [67]:

print(f"{'País':<20}{'Total de Nomes Testados':<25}{'Acertos':<10}{'Erros':<10}{'Acurácia'}")
print("-" * 75)
print(f"{'Portugal':<20}{total_nomes_portugal:<25}{acertos_portugal:<10}{erros_portugal:<10}{acuracia_portugal:.4f}")
print(f"{'Espanha':<20}{total_nomes_espanha:<25}{acertos_espanha:<10}{erros_espanha:<10}{acuracia_espanha:.4f}")
print(f"{'Japão':<20}{total_nomes_japao:<25}{acertos_japao:<10}{erros_japao:<10}{acuracia_japao:.4f}")
print(f"{'Alemanha':<20}{total_nomes_alemanha:<25}{acertos_alemanha:<10}{erros_alemanha:<10}{acuracia_alemanha:.4f}")
print(f"{'Brasil ':<20}{total_nomes_feminino_br+total_nomes_masculino_br:<25}{acertos_feminino_br+acertos_masculino_br:<10}{erros_feminino_br+erros_masculino_br:<10}{(acuracia_masculino_br+acuracia_feminino_br)/2:.4f}")

País                Total de Nomes Testados  Acertos   Erros     Acurácia
---------------------------------------------------------------------------
Portugal            102                      83        19        0.8137
Espanha             83                       72        11        0.8675
Japão               73                       54        19        0.7397
Alemanha            95                       86        9         0.9053
Brasil              200                      163       37        0.8150


De acordo com a tabela podemos perceber que o país que mais acertamos o genero foi Alemanha, seguido da Espanha depois Brasil, Portugal e por fim Japão. 
